The name of this notebook has been changed, please refer to [i_introduction.ipynb](https://github.com/marketpsych/marketpsych/blob/main/notebooks/i_introduction.ipynb) for the original notebook.   
   
We apologise for any inconvenience.